Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1253]:
from itertools import combinations
from collections import namedtuple, defaultdict
from copy import deepcopy
from random import choice
from tqdm.auto import tqdm

In [1254]:
# Define the State namedtuple
State = namedtuple('State', ['x', 'o'])
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]

def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()

def win(elements):
    """Checks if elements are winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(pos: State):
    """Customized state evaluation"""
    # Check if X wins
    if win(pos.x):
        return 1  # Positive value if X wins
    # Check if O wins
    elif win(pos.o):
        return -1  # Negative value if O wins
    else:
        return 0  # Return 0 for a draw

def choose_action(state, available_actions, q_values):
    # Create a hashable representation of the current state
    hashable_state = (tuple(sorted(state.x)), tuple(sorted(state.o)))

    # Get Q-values for the current state from the Q-table
    q_values_state = q_values.get(hashable_state, {})

    # If no Q-values are available, choose a random action
    if not q_values_state:
        return choice(available_actions)

    # Select the action with the maximum Q-value
    available_q_values = {action: q_values_state.get(action, 0) for action in available_actions}
    return max(available_q_values, key=available_q_values.get)


# Reiforcement Learning Strategy

In [1255]:
# Initialize a global cache for memoization
from copy import deepcopy

min_cache = {}
max_cache = {}

def minmax(state, player, maximize):
    cache = max_cache if player == maximize else min_cache
    
    possible = list(set(range(1, 10)) - state.x - state.o)

    state_key = (tuple(sorted(state.x)), tuple(sorted(state.o)))
    if state_key in cache:
        return cache[state_key]
    
    over = state_value(state)
    if over != 0 or not possible:
        if maximize == "o":
            over = -over
        return None, over
    
    scores = []
    for i, move in enumerate(possible):
        new_g = deepcopy(state)
        if player == "x":
            new_g.x.add(move)
            scores.append((i, minmax(new_g, "o", maximize)[1]))
        else:
            new_g.o.add(move)
            scores.append((i, minmax(new_g, "x", maximize)[1]))
    
    if player == maximize:  # max player
        best = max(scores, key=lambda x: x[1])
    else:
        best = min(scores, key=lambda x: x[1])
    
    choice = possible[best[0]]
    cache[state_key] = (choice, best[1])
    return choice, best[1]

def backpropagation(Qtable: dict, states: list, actions: list, reward, learning_rate, gamma, default=0):
    # Initialize the next state as None
    next_state = None
    # Iterate over the reversed list of states and actions
    for act, state in zip(reversed(actions), reversed(states)):
        # Calculate the maximum Q-value for the next state
        fut_max = max(Qtable.get(next_state, {}).values()) if next_state and Qtable.get(next_state) else default
        # Update the Q-value for the current state and action
        Qtable[state][act] = Qtable[state].get(act, 0) + learning_rate * (reward + gamma * fut_max - Qtable[state].get(act, 0))
        # Update the next state for the next iteration
        next_state = state
    
    return Qtable


# Training phase using Random and MinMax

In [1256]:
def q_learning(Qtable, alpha, gamma):
    # Initialize an empty state for the tic-tac-toe game
    state = State(set(), set())
    # Define the set of available moves
    available = set(range(1, 10))

    states = []  # List to store states encountered during the game
    actions = []  # List to store actions taken during the game
    first_player = choice((0, 1))  # Randomly choose the first player
    while available:
        # Choose action based on Q-values for the first player, random for the second player
        if first_player == 0:
            action, _ = minmax(state, "x", "x")
        else:
            action = choice(list(available))

        # Create a copy of the current state
        next_state = State(state.x.copy(), state.o.copy())
        # Update the state with the chosen action
        next_state.x.add(action) if first_player == 0 else next_state.o.add(action)
        # Remove the chosen action from the set of available actions
        available.remove(action)

        # Calculate the reward for the next state
        

        # Add the current state and action to the lists
        states.append((tuple(sorted(state.x)), tuple(sorted(state.o))))
        actions.append(action)

        # Update the current state based on the chosen action
        state.x.add(action) if first_player == 0 else state.o.add(action)

        # Check for a win or if there are no more available moves
        if win(state.x) or win(state.o) or not available:
            break

        first_player = 1 - first_player
       
    # Perform backpropagation to update Q-values based on the game trajectory
    reward = state_value(state)
    Qtable = backpropagation(Qtable, states, actions, reward, alpha, gamma)
    return Qtable

# Initialize Q-values dictionary
q_values = defaultdict(lambda: defaultdict(float))
# Set learning rate (alpha) and discount factor (gamma)
alpha = 0.1
gamma = 0.9

# Run the Q-learning algorithm for a specified number of steps
for steps in tqdm(range(20_000)):
    q_values = q_learning(q_values, alpha, gamma)


  0%|          | 0/20000 [00:00<?, ?it/s]

100%|██████████| 20000/20000 [00:02<00:00, 9399.39it/s] 


In [1257]:
def play_game(q_values):
    # Initialize an empty state for the tic-tac-toe game
    state = State(set(), set())
    # Define the set of available moves
    available = set(range(1, 9 + 1))

    # Randomly choose the first player (0 or 1)
    first_player = choice((0,1))

    while available and state_value(state) == 0:
        if first_player == 0:
            # If it's the first player's turn, choose the action using Q-values
            x = choose_action(state, list(available), q_values)
            state.x.add(x)
        else:
            # If it's the second player's turn, choose a random action
            o = choice(list(available))
            #It is possible to challenge also the min max
            #o, _ = minmax(state, "o", "x")
            state.o.add(o)

        # Remove the chosen action from the set of available actions
        available.remove(x if first_player == 0 else o)
        # Check if the first player wins
        if win(state.x):
            # Return 1 if the first player wins
            return 1
        # Check if the second player wins
        if win(state.o):
            # Return -1 if the second player wins
            return -1
        # Check if the game is a draw
        if not available:
            # Return 0 if the game is a draw
            return 0

        # Switch the player for the next turn
        first_player = 1 - first_player

# Set the number of games to simulate
num_games = 10000
count_q = 0
count_r = 0
count_d = 0

# Run the simulation for the specified number of games
for _ in tqdm(range(num_games)):
    res = play_game(q_values)
    if res == 1:
        count_q += 1
    elif res == -1:
        count_r += 1
    else:
        count_d += 1

# Print the results of the simulation
print(f"Percentage of wins for the Q-learning agent: {(count_q/num_games)*100:.2f}%")
print(f"Percentage of wins for the opponent: {(count_r/num_games)*100:.2f}%")
print(f"Percentage of draws: {(count_d/num_games)*100:.2f}%")


100%|██████████| 10000/10000 [00:00<00:00, 20045.93it/s]

Percentage of wins for the Q-learning agent: 81.17%
Percentage of wins for the opponent: 2.03%
Percentage of draws: 16.80%
